# Projet Python ENSAE

## Tokenisation d'actifs immobiliers

Lien projet: https://hureauxarnaud.medium.com/projet-estimateur-de-prix-dun-bien-immobilier-bas%C3%A9-sur-du-machine-learning-ae578fdacaca

Code : https://colab.research.google.com/drive/1bU1Nx0HcLSSf1U6BVvsiYFfrJeKnhIW2?usp=sharing#scrollTo=GWGxYN3LKLu8   

# Etape 0 : packages

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.neighbors import BallTree

In [ ]:
import requests

url = 'https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip'
r = requests.get(url)
with open("pynsee.zip" , 'wb') as zipfile:
    zipfile.write(r.content)

!pip install --ignore-installed pynsee.zip
!pip install python-Levenshtein

In [62]:
# Get the development version from GitHub
# git clone https://github.com/InseeFrLab/Py-Insee-Data.git
# cd Py-Insee-Data
# pip install .
# pip install --ignore-installed pynsee.zip
# pip install xlrd
# pip install python-Levenshtein

In [3]:
# import pynsee
# import pynsee.download
# from pynsee.download import telechargerDonnees

In [4]:
# df_city_2015 = telechargerDonnees("FILOSOFI_COM", date = "2015")

In [5]:
# Variable nombre de personnes: `NBPERSMENFISC16`
# df_city_2015.sample(5)

# Etape 1 : preprocessing

### Dataset :

Nous importons le dataset « Demandes de valeurs foncières » (DVF), publié par la DGFiP, permet de connaître les transactions immobilières intervenues au cours des cinq dernières années sur le territoire métropolitain et les DOM-TOM, à l’exception de l’Alsace, de la Moselle et de Mayotte. Les données contenues sont issues des actes notariés et des informations cadastrales.

Fichiers 2017-2020 : https://files.data.gouv.fr/geo-dvf/latest/

### Filtrage du dataset :

Nous allons filtrer notre dataset de la manière suivante :

- nous ne prenons que les types de mutation “Vente” et “Vente en l’état futur d’achèvement”, pour éviter n’inclure dans notre modèle des ventes de terrains / ventes issus d’adjudication.

- nous ne prenons que les ventes dont le nombre de m² bâti est renseigné et différent de zéro.

- nous ne prenons que les ventes dont le prix de ventes est renseigné et différent de zéro.

### Problème de filtrage :

Ensuite, nous avons un problème : le dataset affiche le même prix de vente global à chaque lot.

En effet, d'après la notice descriptive : 
* Les lots, notion juridique immobilière, sont définis dans un état descriptif de division (EDD) et dans les documents de mutation. Ils permettent d’identifier une partie d’un immeuble et ainsi d’y associer un droit de propriété spécifique (propriété, usufruit, etc.).
* Les locaux, notion fiscale, identifient les mêmes parties d’un immeuble pour les besoins de la taxe foncière et de la taxe d’habitation en regroupant plusieurs lots.
* La correspondance entre le découpage en lots et en locaux n’est pas retracée.
Donc, quand une disposition comporte plusieurs locaux ou plusieurs natures de culture, le fichier de restitution comporte autant de lignes qu’il y a de locaux ou de nature de culture concernés par la mutation. Ainsi, pour une même publication, il peut y avoir 1 à n ligne(s) de restitution. Les données génériques (ainsi que le prix) sont alors répétées sur chaque ligne.

Voici les types de locaux différents : 
['Dépendance', 'Appartement', 'Maison', nan, 'Local industriel. commercial ou assimilé']
On ne peut pas juste enlever les "Dépendances". En effet, ces dernières peuvent avoir un effet (haussier) sur le prix de vente.

Par ailleurs, une maison avec un jardin fait l'objet de deux lignes ayant la même valeur foncière mais des surfaces différentes (celle de la maison, celle du jardin) : suivant la colonne nature_culture ([nan, 'sols', "terrains d'agrément", 'taillis simples', 'terrains a bâtir', 'eaux', 'landes', 'taillis sous futaie', 'prés', 'terres', 'jardins', 'peupleraies', 'vignes', 'bois', 'vergers', 'carrières', 'futaies résineuses', 'pâtures', 'futaies feuillues', 'futaies mixtes', 'chemin de fer', 'oseraies', 'pacages', 'prés plantes', 'terres plantées', 'landes boisées', 'herbages', "prés d'embouche"]).

Il conviendra donc d'agréger dans une ligne ces informations :
* l'actif immobilier a-t-il une dépendance ? si oui, de combien de m2 ?
* la maison a-t-elle un jardin ? si oui, de combien de m2 ?

### Dataset final :

Enfin, nous ne prenons que les colonnes suivantes :
- Date de vente/mutation
- Nature mutation (pour séparer les ventes en VEFA et les ventes classiques)
- Valeur foncière (prix de vente)
- Colonnes liées à l’adresse (pour nous permettre de localiser le bien)
- Adresse
- Code Postal
- Type local (maison/appartement/Local commercial/Dépendance etc)
- Surface réelle bâtie (nb de mètre carré du bien bâti)
- Surface terrain (nb de mètre carré du terrain associé au bien)

In [14]:
table = pd.read_csv('data/valeursfoncieres-2018.csv', sep = ',')

C:\Users\victo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10,12,14,16,17,18,20,22,26,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
table.head(7)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
1,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Appartement,73.0,4.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
2,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,S,sols,NaN,NaN,949.0,5.408041,46.255562
3,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,51.0,2.0,S,sols,NaN,NaN,949.0,5.408041,46.255562
4,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
5,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,51.0,2.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
6,2018-3,2018-01-04,1,Vente,90000.0,NaN,NaN,LE DRUILLET,B034,1380.0,...,NaN,NaN,NaN,S,sols,NaN,NaN,278.0,4.941705,46.264004


In [20]:
print("* Différentes valeurs dans 'nombre_lots' :")
display(table['nombre_lots'].unique())
print("* Différentes valeurs dans 'type_local' :")
display(table['type_local'].unique())
print("* Différentes valeurs dans 'code_type_local' :")
display(table['code_type_local'].unique())
print("* Différentes valeurs dans 'nature_culture' :")
display(table['nature_culture'].unique())
print("* Comparaison lignes 2 / 3 / 4 / 5 :")
display(table.loc[2])
display(table.loc[3])
display(table.loc[4])
display(table.loc[5])

* Différentes valeurs dans 'nombre_lots' :


array([  1,   2,   0,   3,   5,   4,   6,   7,  20,  16,  13,   9,   8,
        15,  10,  12,  17,  11,  22,  56,  18,  45,  89,  19,  14,  48,
        21, 119,  50,  27,  61,  25,  26,  37,  34, 101,  30,  47,  24,
        90,  69,  33,  32,  46,  75,  31,  35,  36,  52,  41,  57,  40,
        60,  23,  58,  42,  43, 112,  44, 330,  38,  39,  29,  28,  76,
        49, 131,  72, 116, 121,  65,  79,  53, 136,  66,  88,  64, 100,
       198, 120, 223,  84], dtype=int64)

* Différentes valeurs dans 'type_local' :


array(['Dépendance', 'Appartement', 'Maison', nan,
       'Local industriel. commercial ou assimilé'], dtype=object)

* Différentes valeurs dans 'code_type_local' :


...

* Comparaison lignes 2 / 3 / 4 / 5 :


id_mutation                                 2018-2
date_mutation                           2018-01-04
numero_disposition                               1
nature_mutation                              Vente
valeur_fonciere                           239300.0
adresse_numero                                 4.0
adresse_suffixe                                NaN
adresse_nom_voie                RUE DE LA BARMETTE
adresse_code_voie                             0025
code_postal                                 1250.0
code_commune                                  1095
nom_commune                       Nivigne et Suran
code_departement                                 1
ancien_code_commune                            NaN
ancien_nom_commune                             NaN
id_parcelle                         01095000AH0186
ancien_id_parcelle                             NaN
numero_volume                                  NaN
lot1_numero                                    NaN
lot1_surface_carrez            

id_mutation                                 2018-2
date_mutation                           2018-01-04
numero_disposition                               1
nature_mutation                              Vente
valeur_fonciere                           239300.0
adresse_numero                                 4.0
adresse_suffixe                                NaN
adresse_nom_voie                RUE DE LA BARMETTE
adresse_code_voie                             0025
code_postal                                 1250.0
code_commune                                  1095
nom_commune                       Nivigne et Suran
code_departement                                 1
ancien_code_commune                            NaN
ancien_nom_commune                             NaN
id_parcelle                         01095000AH0186
ancien_id_parcelle                             NaN
numero_volume                                  NaN
lot1_numero                                    NaN
lot1_surface_carrez            

id_mutation                                  2018-2
date_mutation                            2018-01-04
numero_disposition                                1
nature_mutation                               Vente
valeur_fonciere                            239300.0
adresse_numero                                  4.0
adresse_suffixe                                 NaN
adresse_nom_voie                 RUE DE LA BARMETTE
adresse_code_voie                              0025
code_postal                                  1250.0
code_commune                                   1095
nom_commune                        Nivigne et Suran
code_departement                                  1
ancien_code_commune                             NaN
ancien_nom_commune                              NaN
id_parcelle                          01095000AH0186
ancien_id_parcelle                              NaN
numero_volume                                   NaN
lot1_numero                                     NaN
lot1_surface

id_mutation                                  2018-2
date_mutation                            2018-01-04
numero_disposition                                1
nature_mutation                               Vente
valeur_fonciere                            239300.0
adresse_numero                                  4.0
adresse_suffixe                                 NaN
adresse_nom_voie                 RUE DE LA BARMETTE
adresse_code_voie                              0025
code_postal                                  1250.0
code_commune                                   1095
nom_commune                        Nivigne et Suran
code_departement                                  1
ancien_code_commune                             NaN
ancien_nom_commune                              NaN
id_parcelle                          01095000AH0186
ancien_id_parcelle                              NaN
numero_volume                                   NaN
lot1_numero                                     NaN
lot1_surface

In [163]:
# Les fichiers sont renommés valeursfoncieres-(année)

def preprocessing(year):
    # path = 'Users\victo\Desktop\ENSAE\Python\tokenisation-immo\valeursfoncieres-'+str(year)+'.csv'
    table = pd.read_csv('data/valeursfoncieres-'+ str(year) +'.csv', sep = ',')
    
    a = [2, 4, 5, 6, 7, 8, 9, 10, 12, 29, 31, 32, 38, 39, 40] 
    # vecteur des n° des colonnes qui nous intéressent
    
    a = [k - 1 for k in a]
    
    table = table[table.columns[a]]

    ventes_types = list(table[table.columns[1]].unique())
    
    table = table[(table['nature_mutation'] == 'Vente') | 
                  (table['nature_mutation'] == "Vente en l'état futur d'achèvement")]

    table = table[(table['nombre_lots'] == 0) | (table['nombre_lots'] == 1)]

    table = table[table['surface_reelle_bati']!=0]

    table = table[table['surface_reelle_bati'].notna()]

    table = table[table['valeur_fonciere'].notna()]
    
    table['adresse_numero'] = table['adresse_numero'].fillna('0').astype(int)
    table['adresse_suffixe'] = table['adresse_suffixe'].fillna(' ')
    table['adresse_code_voie'] = table['adresse_code_voie'].fillna(' ')
    table['adresse_nom_voie'] = table['adresse_nom_voie'].fillna(' ')
    table['code_postal'] = table['code_postal'].fillna('0').astype(int)
    table['nom_commune'] = table['nom_commune'].fillna(' ')

    table['Adresse'] = table['adresse_numero'].astype(str) + ' ' + table['adresse_suffixe'] + ' ' + table['adresse_code_voie'] + ' ' + table['adresse_nom_voie'] + ' ' + table['nom_commune'] + ' ' + table['code_postal'].astype(str) + ' ' + 'France'    
    
    table.reset_index(drop = True, inplace = True)
  
    return table

In [168]:
test = preprocessing(2017)

C:\Users\victo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (10,12,14,16,17,18,20,22,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [173]:
display(len(test))
test.head()

1140324

,date_mutation,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,nombre_lots,type_local,surface_reelle_bati,surface_terrain,longitude,latitude,Adresse
0,2017-01-03,Vente,258000.0,11,,IMP DES PINSONS,0384,1000,Saint-Denis-lès-Bourg,0,Appartement,22.0,655.0,5.205776,46.197455,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...
1,2017-01-03,Vente,258000.0,11,,IMP DES PINSONS,0384,1000,Saint-Denis-lès-Bourg,0,Appartement,22.0,655.0,5.205776,46.197455,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...
2,2017-01-03,Vente,258000.0,11,,IMP DES PINSONS,0384,1000,Saint-Denis-lès-Bourg,0,Appartement,120.0,655.0,5.205776,46.197455,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...
3,2017-01-05,Vente,175050.0,11,,SAINT MICHEL,B244,1370,Val-Revermont,0,Maison,99.0,471.0,5.361589,46.274684,11 B244 SAINT MICHEL Val-Revermont 1370 France
4,2017-01-06,Vente,162000.0,6,,RUE LOUIS BLERIOT,2330,1000,Bourg-en-Bresse,1,Appartement,69.0,NaN,5.233978,46.211926,6 2330 RUE LOUIS BLERIOT Bourg-en-Bresse 100...


In [174]:
test.drop_duplicates(subset = ['date_mutation', 'valeur_fonciere', 'adresse_nom_voie'], keep='last')
test.head()

,date_mutation,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,nombre_lots,type_local,surface_reelle_bati,surface_terrain,longitude,latitude,Adresse
0,2017-01-03,Vente,258000.0,11,,IMP DES PINSONS,0384,1000,Saint-Denis-lès-Bourg,0,Appartement,22.0,655.0,5.205776,46.197455,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...
1,2017-01-03,Vente,258000.0,11,,IMP DES PINSONS,0384,1000,Saint-Denis-lès-Bourg,0,Appartement,22.0,655.0,5.205776,46.197455,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...
2,2017-01-03,Vente,258000.0,11,,IMP DES PINSONS,0384,1000,Saint-Denis-lès-Bourg,0,Appartement,120.0,655.0,5.205776,46.197455,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...
3,2017-01-05,Vente,175050.0,11,,SAINT MICHEL,B244,1370,Val-Revermont,0,Maison,99.0,471.0,5.361589,46.274684,11 B244 SAINT MICHEL Val-Revermont 1370 France
4,2017-01-06,Vente,162000.0,6,,RUE LOUIS BLERIOT,2330,1000,Bourg-en-Bresse,1,Appartement,69.0,NaN,5.233978,46.211926,6 2330 RUE LOUIS BLERIOT Bourg-en-Bresse 100...


In [6]:
# Fonction pour mettre dans 'dicte' les 4 csv après preprocessing

dicte = {}
years = [2017,2018,2019,2020]
for k in years:
    dicte[k-2017] = preprocessing(k)

C:\Users\victo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (10,12,14,16,17,18,20,22,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\victo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (10,12,14,16,17,18,20,22,26,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
output = pd.DataFrame()
for k in range(4):
    output = pd.concat((output,dicte[k]), axis = 0)

In [13]:
output = output[['date_mutation', 'nature_mutation', 'valeur_fonciere', 'type_local', 
                 'surface_reelle_bati', 'surface_terrain', 'Adresse', 'latitude', 'longitude']]
output.sample(10)

,date_mutation,nature_mutation,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,Adresse,latitude,longitude
0,2017-01-03,Vente,258000.0,Appartement,22.0,655.0,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...,46.197455,5.205776
1,2017-01-03,Vente,258000.0,Appartement,22.0,655.0,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...,46.197455,5.205776
2,2017-01-03,Vente,258000.0,Appartement,120.0,655.0,11 0384 IMP DES PINSONS Saint-Denis-lès-Bour...,46.197455,5.205776
3,2017-01-05,Vente,175050.0,Maison,99.0,471.0,11 B244 SAINT MICHEL Val-Revermont 1370 France,46.274684,5.361589
4,2017-01-06,Vente,162000.0,Appartement,69.0,NaN,6 2330 RUE LOUIS BLERIOT Bourg-en-Bresse 100...,46.211926,5.233978
...,...,...,...,...,...,...,...,...,...
1060421,2020-12-16,Vente,1937500.0,Appartement,64.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486
1060422,2020-12-16,Vente,1937500.0,Appartement,40.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486
1060423,2020-12-16,Vente,1937500.0,Appartement,33.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486
1060424,2020-12-16,Vente,1937500.0,Appartement,27.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486


In [14]:
# fonction pour exporter le final en div=10 petits fichiers

div=10
for k in range(div):
    name='final-'+str(1+k)+'.csv'
    df=output[k*round(len(output)/div):(k+1)*round(len(output)/div)]
    df.to_csv(name,index=False,header=True)

# Etape 2 : création du modèle pour obtenir le prix moyen du voisinage (BallTree)

### 2.1. Importation des données

data_vf : base de données de l'INSEE sur l'historique des valeurs foncières

data_communes : base de données de l'INSEE sur les communes françaises

In [99]:
# importation de data_vf
div = 10
data_vf = pd.read_csv('data/final-1.csv', sep = ',')
for k in range(1, div):
    name = 'data/final-' + str(1 + k) + '.csv'
    data_vf = pd.concat([data_vf, pd.read_csv(name, sep = ',')])
    
# importation de data_communes
data_communes = pd.read_csv('communes_insee.csv', sep = ';')

# importation de data_regions
data_regions = pd.read_csv('communes-departement-region.csv', sep = ',')

C:\Users\victo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,15,100) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 2.2. Visualisation des DataFrames

In [131]:
# Visualisation de data_vf
print("Nombre de lignes de data data_vf :")
print(len(data_vf))
print("Nombre de colonnes de data data_vf :")
print(len(data_vf.columns))
print("Visualisation de data_vf :")
data_vf.sample(3)

Nombre de lignes de data data_vf :
4570326
Nombre de colonnes de data data_vf :
9
Visualisation de data_vf :


,date_mutation,nature_mutation,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,Adresse,latitude,longitude
441354,2017-04-25,Vente,88000.0,Maison,162.0,162.0,4 7680 CITE TATTEGRAIN Amiens 80000 France,49.906171,2.275168
91820,2017-07-31,Vente,8000000.0,Appartement,38.0,8399.0,14 3860 RUE DE LA POTERIE Corbeil-Essonnes 9...,48.615961,2.484637
54059,2019-03-26,Vente,1860000.0,Local industriel. commercial ou assimilé,826.0,2751.0,2 0825 RUE DU CARRE NORGANDS Sautron 44880 F...,47.256058,-1.648939


In [132]:
# Visualisation de data_communes
print("Nombre de lignes de data data_communes :")
print(len(data_communes))
print("Nombre de colonnes de data data_communes :")
print(len(data_communes.columns))
print("Visualisation de data_communes :")
data_communes["REG"].unique()

Nombre de lignes de data data_communes :
36677
Nombre de colonnes de data data_communes :
101
Visualisation de data_communes :


array([82, 22, 83, 93, 21, 73, 91, 25, 54, 24, 74, 26, 53, 72, 43, 23, 94,
       52, 41, 31, 42, 11,  1,  2,  3,  4], dtype=int64)

In [134]:
# Visualisation de data_regions
print("Nombre de lignes de data data_regions :")
print(len(data_regions))
print("Nombre de colonnes de data data_regions :")
print(len(data_regions.columns))
print("Visualisation de data_regions :")
display(data_regions.sample(3))
len(data_regions["code_region"].unique())

Nombre de lignes de data data_regions :
39201
Nombre de colonnes de data data_regions :
15
Visualisation de data_regions :


,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
593,2137,BUSSIARES,2810,BUSSIARES,NaN,49.076591,3.250262,137.0,NaN,Bussiares,Bussiares,2,Aisne,32.0,Hauts-de-France
22107,54207,FRAISNES EN SAINTOIS,54930,FRAISNES EN SAINTOIS,NaN,48.374206,6.060920,207.0,NaN,Fraisnes-en-Saintois,Fraisnes-en-Saintois,54,Meurthe-et-Moselle,44.0,Grand Est
36781,86217,ST CHRISTOPHE,86230,ST CHRISTOPHE,NaN,46.925281,0.374730,217.0,NaN,Saint-Christophe,Saint-Christophe,86,Vienne,75.0,Nouvelle-Aquitaine


19

In [135]:
df_co2 = pd.read_csv("https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert")
df_co2.sample(3)

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire
23240,60100,BRASSEUSE,1538.331660,NaN,NaN,2394.771227,10.976179,202.491966,594.364348,39.076745,23014.428150,39.718386
30803,76734,VERGETOT,1296.257937,NaN,NaN,365.890566,57.261273,9.418231,27.644853,170.448959,1702.040285,207.205557
8512,24425,SAINT-JEAN-DE-COLE,694.270708,NaN,NaN,505.694625,157.602559,87.118636,255.714894,210.327122,654.762001,174.186658


2.3. Création d'une BDD inédite

In [159]:
data_vf[data_vf.index.duplicated()]

,date_mutation,nature_mutation,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,Adresse,latitude,longitude
0,2017-03-03,Vente,190000.0,Maison,126.0,431.0,7 0046 RUE DES NOYERS Fossé 41330 France,47.633330,1.282117
1,2017-03-08,Vente,80000.0,Maison,40.0,NaN,24 0390 RUE DU GAZOMETRE Montrichard Val de ...,47.341854,1.179088
2,2017-03-08,Vente,80000.0,Maison,30.0,NaN,24 B 0390 RUE DU GAZOMETRE Montrichard Val de ...,47.341854,1.179088
3,2017-03-21,Vente,95000.0,Local industriel. commercial ou assimilé,271.0,1200.0,215 0070 RUE DE CHAMP CHARDON Mont-près-Cham...,47.556513,1.443888
4,2017-03-23,Vente,163000.0,Maison,79.0,626.0,5 0104 RUE DU 8 MAI 1945 Cour-Cheverny 41700...,47.513564,1.453183
...,...,...,...,...,...,...,...,...,...
457024,2020-12-16,Vente,1937500.0,Appartement,64.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486
457025,2020-12-16,Vente,1937500.0,Appartement,40.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486
457026,2020-12-16,Vente,1937500.0,Appartement,33.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486
457027,2020-12-16,Vente,1937500.0,Appartement,27.0,447.0,24 1748 RUE CHANOINESSE Paris 4e Arrondissem...,48.854301,2.350486


In [158]:
data_new = data_vf.copy()
data_new.reset_index(drop=True, inplace=True)

# prix_m2
data_new['prix_m2'] = data_vf['surface_terrain'] / data_vf['valeur_fonciere']

# cp
data_new['cp'] = [x[-12:][:5] for x in data_vf["Adresse"].values]
# data_new.set_index('cp')

# dep
data_new['dep'] = [x[:2] for x in data_new['cp'].values]

# region


ValueError: cannot reindex from a duplicate axis

In [157]:
data_new

1    11   0384 IMP DES PINSONS Saint-Denis-lès-Bour...
1    24   0390 RUE DU GAZOMETRE Montrichard Val de ...
1    5013   0058 RTE DE SAINT PONS Anglès 81260 France
1    16   A005 RES DU GROS CHENE Bazincourt-sur-Ept...
1    140   0228 RUE CLEMENT ADER Beaumont 63110 France
1          14   0210 RUE DU BROUAGE Chauny 2300 France
1     4   0320 RUE JOSEPH VENDRE Heyrieux 38540 France
1    28   0280 RUE DE VILLEMARECHAL Nanteau-sur-Lun...
1    17 T 0887 AV DE L ILE MADAME Port-des-Barques ...
1      102   2560 RUE LEON GAMBETTA Douai 59500 France
Name: Adresse, dtype: object

In [150]:
# code_region
x = data_new["cp"].loc[2]
x

2     1000
2    41400
2    81260
2    27700
2    63370
2     2300
2    38440
2    77130
2    17550
2    59950
Name: cp, dtype: object

In [ ]:

data_regions["nom_region"].iloc[data_new[]]

In [ ]:
regions = 13
appart_old[‘Distance moyenne’]=np.zeros(len(appart_old))
appart_old[‘Indices voisins’]=np.zeros(len(appart_old))
models={}
regions=appart_old.Région.unique()
for k in range(len(regions)):
    name=’appart_’+regions[k]
    data=appart_old[appart_old.Région==regions[k]]
    data=data.reset_index(drop=True)
    models[k]=BallTree(data[[‘latitude_r’, ‘longitude_r’]].values, leaf_size=2, metric=’haversine’)
    save_obj(models[k], name)